# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

import geoviews.feature as gf
import xarray as xr
from cartopy import crs





In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot_1
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity"
)


# Display the map plot_1
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
perfect_df = city_data_df[(abs(city_data_df['Lat']) <= 50) & (city_data_df['Humidity'] <= 80)]

# Drop any rows with null values
perfect_df.dropna()

# Display sample data
perfect_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230
6,6,albany,42.6001,-73.9662,10.36,74,29,2.10,US,1666108231
...,...,...,...,...,...,...,...,...,...,...
566,566,nhulunbuy,-12.2333,136.7667,26.34,78,88,0.00,AU,1666108559
568,568,road town,18.4167,-64.6167,31.02,70,40,4.12,VG,1666108560
574,574,canchungo,12.0672,-16.0333,35.07,61,67,2.91,GW,1666108565
575,575,iquique,-20.2208,-70.1431,17.02,68,6,3.50,CL,1666108566


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = perfect_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.reset_index()

,index,City,Country,Lat,Lng,Humidity,Hotel Name
0,0,faya,SA,18.3851,42.4509,35,
1,2,new norfolk,AU,-42.7826,147.0587,58,
2,3,jamestown,US,42.0970,-79.2353,77,
3,4,lanzhou,CN,36.0564,103.7922,48,
4,6,albany,US,42.6001,-73.9662,74,
...,...,...,...,...,...,...,...
291,566,nhulunbuy,AU,-12.2333,136.7667,78,
292,568,road town,VG,18.4167,-64.6167,70,
293,574,canchungo,GW,12.0672,-16.0333,61,
294,575,iquique,CL,-20.2208,-70.1431,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # Get the city's name
    city = row["City"]
    
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionary and convert to JSON
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: No hotel found
new norfolk - nearest hotel: Woodbridge on the Derwent
jamestown - nearest hotel: DoubleTree Jamestown
lanzhou - nearest hotel: 西园宾馆
albany - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
trairi - nearest hotel: Pousada Villa Aurora
avarua - nearest hotel: Paradise Inn
guiratinga - nearest hotel: No hotel found
rikitea - nearest hotel: Chez Bianca & Benoit
constitucion - nearest hotel: Hostal Alameda
bambous virieux - nearest hotel: Otentic Eco Tent Experience
zangakatun - nearest hotel: No hotel found
saint-louis - nearest hotel: No hotel found
midwest city - nearest hotel: No hotel found
cape town - nearest hotel: Townhouse Hotel
naberera - nearest hotel: No hotel found
drochia - nearest hotel: Hotel Confort
tomatlan - nearest hotel: Hotel Hacienda Vieja Tomatlan
sao joao da barra - nearest hotel: Pousada Mediterrâneo
ancud - nearest hotel: Cabañas
colliervi

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,No hotel found
2,new norfolk,AU,-42.7826,147.0587,58,Woodbridge on the Derwent
3,jamestown,US,42.0970,-79.2353,77,DoubleTree Jamestown
4,lanzhou,CN,36.0564,103.7922,48,西园宾馆
6,albany,US,42.6001,-73.9662,74,No hotel found
...,...,...,...,...,...,...
566,nhulunbuy,AU,-12.2333,136.7667,78,No hotel found
568,road town,VG,18.4167,-64.6167,70,No hotel found
574,canchungo,GW,12.0672,-16.0333,61,Hotel Pereira
575,iquique,CL,-20.2208,-70.1431,68,Hostal Vivar


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ['Country','Hotel Name']
)

# Display the map
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Hotel Name)